In [1]:
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

In [7]:
# Used the inbuilt tool of wikipedia
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1,
                                       doc_content_chars_max=250)

wiki = WikipediaQueryRun(api_wrapper= api_wrapper_wiki)
wiki

WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/home/aditya/anaconda3/envs/langchain/lib/python3.10/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250))

In [9]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1,
                                    doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv

ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250))

In [38]:
wiki.invoke("Google")

'Page: Google\nSummary: Google LLC ( , GOO-gəl) is an American multinational corporation and technology company focusing on online advertising, search engine technology, cloud computing, computer software, quantum computing, e-commerce, consumer electr'

In [10]:
tools = [wiki,arxiv]

In [11]:
## Custom tools

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [22]:
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_APIKEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [17]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb = FAISS.from_documents(documents,OpenAIEmbeddings())
retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7ba7ec2b6f20>, search_kwargs={})

In [ ]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(retriever,
                                       "langsmith_search",
                                       "Search any info about langsmith")
retriever_tool.name

In [21]:
tools = [wiki,arxiv,retriever_tool]

In [24]:
from langchain_groq import ChatGroq
llm = ChatGroq(model="Llama3-8b-8192")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7ba7c53f8a00>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7ba7c53faa10>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [25]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

/home/aditya/anaconda3/envs/langchain/lib/python3.10/site-packages/langsmith/client.py:277: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [27]:
from langchain.agents import create_openai_tools_agent

agent = create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [29]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [30]:
agent_executor.invoke({"input":"tell me about langsimth"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

LangSmith + LangChain OSSLangSmith is framework-agnostic — it can be used with or without LangChain's open source frameworks langchain and langgraph.If you are using either of these, you can enable LangSmith tracing with a single environment variable.
For more see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.
Observability​
Observability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to debug.
This is where LangSmith can help! LangSmith has LLM-native observability, allowing you to get meaningful insights from your application. LangSmith’s observability features have you covered throughout all stages of application development - from prot

{'input': 'tell me about langsimth',
 'output': 'Based on the output from the tool, it seems that LangSmith is a platform for building production-grade LLM (Large Language Model) applications. It provides features such as observability, evaluation, and prompt engineering to help developers closely monitor and evaluate their applications, and iterate on prompts to optimize their performance.\n\nSome of the key features of LangSmith include:\n\n* Observability: LangSmith provides LLM-native observability, allowing developers to get meaningful insights from their application and debug issues more easily.\n* Evaluation: LangSmith allows developers to evaluate their application over production traffic, and provides features for scoring application performance and collecting human feedback on data.\n* Prompt Engineering: LangSmith provides tools for prompt engineering, enabling developers to find the perfect prompt for their application and iterate on prompts to optimize performance.\n\nOver

In [34]:
agent_executor.invoke({"input":"Tell me about Ye maya chesave"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Ye Maya Chesave'}`


Page: Ye Maaya Chesave
Summary: Ye Maaya Chesave (transl. What magic have you done?) is a 2010 Indian Telugu-language romantic drama film written and directed by Gautham Vasudev Menon. The film stars Naga Chaitanya and Samantha in her debut as lead. 
Invoking: `wikipedia` with `{'query': 'Naga Chaitanya'}`


Page: Naga Chaitanya
Summary: Akkineni Naga Chaitanya (Telugu: [akːineːni naːɡa tʃaitaɲa]; born 23 November 1986) is an Indian actor who works in Telugu cinema. Chaitanya is a recipient of several accolades including one Filmfare Award South and one 
Invoking: `wikipedia` with `{'query': 'Samantha'}`


Page: Samantha
Summary: Samantha is an English feminine given name in use since the 17th century that is of uncertain derivation. It is now in popular use worldwide due to various popular culture influences.


Invoking: `langsmith_search` with `{'query': 'Samantha'}`


Get started wit

APIError: Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.

In [32]:
agent_executor.invoke({"input":"What's the paper 1706.03762 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on c
Invoking: `wikipedia` with `{'query': 'Attention Is All You Need'}`


Page: Attention Is All You Need
Summary: "Attention Is All You Need" is a 2017 landmark research paper in machine learning authored by eight scientists working at Google. The paper introduced a new deep learning architecture known as the transformer,
Invoking: `wikipedia` with `{'query': 'Attention Is All You Need'}`


Page: Attention Is All You Need
Summary: "Attention Is All You Need" is a 2017 landmark research paper in machine learning authored by eight scientists working at Google. The paper introduced a new deep learning architecture known as the transformer,The paper "Atten

{'input': "What's the paper 1706.03762 about?",
 'output': 'The paper "Attention Is All You Need" was about the transformer architecture, and we\'ve already discussed that. Since we\'ve already covered the topic, I\'ll respond directly:\n\nThe paper "Attention Is All You Need" introduced the transformer architecture, which revolutionized the field of natural language processing and has since been widely adopted in many applications.'}

In [43]:
load_dotenv()
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_SEARCH_API")

In [45]:
from langchain_tavily import TavilySearch

tavilytool = TavilySearch(
    max_results=5,
    topic="general",
)

In [46]:
tavilytool.invoke("Who won IPL 2024?")

{'query': 'Who won IPL 2024?',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'title': 'Who won IPL 2024? What happened in the final between KKR and SRH?',
   'url': 'https://www.sportingnews.com/in/cricket/news/who-won-ipl-final-2024-kkr-srh/2d2abac1a403fa86e6c5afbd',
   'content': 'Who won IPL 2024? What happened in the final between KKR and SRH? | Sporting News India IPL IPL EXPLORE IPL NBA TEAMS EXPLORE FOOTBALL Cricket IPL NBA Football IPL Who won IPL 2024? Kolkata Knight Riders (KKR) beat Sunrisers Hyderabad (SRH) by eight wickets in the IPL 2024 Final at the MA Chidambaram Stadium in Chennai to lift a third IPL trophy on Sunday. With this result, KKR are now the third most successful team in the history of the league after Mumbai Indians and Chennai Super Kings who have won it five times each. MORE: Relive all the drama of the 2024 IPL Final here — IndianPremierLeague (@IPL) May 26, 2024 IPL Final 2024 result, score Sporting News',
   'score': 0.9521